# Fallacy detector chain

> This module made to make a logical fallacy search tool for LLM-based agent.
> So it should find stuff like ad hominem, self-contradicting statements and so on, given the discussion history and so on.

## Implementation

In [ ]:
#| default_exp fallacy

In [ ]:
#| export
from pino_inferior.core import DATA_DIR, PROMPTS_DIR, OPENAI_API_KEY
from langchain.schema.runnable import RunnableSequence
from langchain.llms.openai import BaseLLM
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains.transform import TransformChain
from langchain.schema.messages import AIMessage, AIMessageChunk
import os
from dataclasses import dataclass
from typing import Union, List, Callable
import json
from datetime import datetime
from pino_inferior.message import Message
import tiktoken

In [ ]:
#| export
FALLACIES_FNAME = os.path.join(DATA_DIR, "fallacies.json")
FALLACIES_PROMPT_DIR = os.path.join(PROMPTS_DIR, "fallacies")

In [ ]:
#| export
INPUT_FALLACIES = "fallacies"
INPUT_HISTORY = "history"
INPUT_CONTEXT = "context"
INPUT_QUERY = "query"

INTERMEDIATE_FALLACIES_STR = "fallacies_str"
INTERMEDIATE_HISTORY_STR = "history_str"
INTERMEDIATE_LAST_AUTHOR = "last_message_author"

OUTPUT_LLM_OUTPUT = "llm_output"
OUTPUT_SHORT_ANSWER = "answer"

LLM_OUTPUT_MARKER = "Therefore"

### Fallacy representation

The following structures represents fallacy description as well as convert it to LLM-readable text.

In [ ]:
#| export
@dataclass
class FallacyExample:
    """
    Example of a logical fallacy
    """
    text: str # Statement
    response: str # Fallacy detector response, explaining why it is a fallacy

    def __str__(self) -> str:
        return f"Example: {self.text}\nExample Response: {self.response}"


@dataclass
class Fallacy:
    """
    Fallacy representation
    """
    name: str # Fallacy name (like "ad hominem" and so)
    description: str # Fallacy description
    example: Union[FallacyExample, None] # Fallacy example

    def __str__(self):
        result = f"# {self.name}\n\n{self.description}"
        if self.example:
            result += "\n\n" + str(self.example)
        return result
    

def read_fallacies(fname: str) -> List[Fallacy]:
    """
    Read the file with fallacies markup
    :param fname: File name. Should contain JSON representing a list of `Fallacy`
    :returns: Fallacies list
    """
    with open(fname, "r", encoding="utf-8") as src:
        data = json.load(src)
    result = []
    for item in data:
        if item.get("example"):
            example = FallacyExample(**item["example"])
        else:
            example = None
        fallacy = Fallacy(name=item["name"], description=item["description"], example=example)
        result.append(fallacy)
    return result

### Prompts

Here I will just read predefined prompt templates:

In [ ]:
#| export
system_prompt = SystemMessagePromptTemplate.from_template_file(
    os.path.join(FALLACIES_PROMPT_DIR, "system.txt"),
    input_variables=[]
)
instruction_prompt = HumanMessagePromptTemplate.from_template_file(
    os.path.join(FALLACIES_PROMPT_DIR, "instruction.txt"),
    input_variables=[INTERMEDIATE_FALLACIES_STR,
                     INTERMEDIATE_HISTORY_STR,
                     INTERMEDIATE_LAST_AUTHOR,
                     INPUT_CONTEXT,
                     INPUT_QUERY]
)
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, instruction_prompt])

### Conversions

#### History & fallacies to strings

Initially message history / fallacy types presented as objects.

We will need to stringify them.

Meanwhile we should as well check lengths and maybe cut history to make sure we will fit into LLM context window.

In [ ]:
#| export
def _stringify(row: dict,
               length_function: Callable[[str], int],
               max_fallacies_length: int,
               max_messages_length: int) -> dict:
    fallacies: List[Fallacy] = row[INPUT_FALLACIES]
    history: List[Message] = row[INPUT_HISTORY] # TODO: cut

    fallacies_str = "\n\n".join(map(str, fallacies))
    while True:
        history_str = "\n\n".join(map(str, history))
        if length_function(history_str) <= max_messages_length:
            break
        else:
            history = history[1:]
    assert len(history) > 0, f"History length became less than {max_messages_length} only after removing all messages"
    assert length_function(fallacies_str) <= max_fallacies_length, f"Too big fallacies representation. Expected up to {max_fallacies_length}, got {length_function(fallacies_str)}"

    return {
        INTERMEDIATE_FALLACIES_STR: fallacies_str,
        INTERMEDIATE_HISTORY_STR: history_str,
    }

#### Extract last user name

We will explicitly tell model to search for issues in the last user messages - we interested in it, aren't we?

In [ ]:
#| export
def _extract_last_user(row: dict) -> dict:
    history: List[Message] = row[INPUT_HISTORY]
    assert len(history) > 0
    return {
        INTERMEDIATE_LAST_AUTHOR: history[-1].author
    }

#### Short answer extraction

After getting a full answer from LLM we should extract the short form.

In [ ]:
#| export
def _extract_answer_from_cot(row: dict) -> dict:
    response: Union[AIMessage, AIMessageChunk] = row[OUTPUT_LLM_OUTPUT]
    text: str = response.content
    text = text.split(LLM_OUTPUT_MARKER)[-1]
    text = text.split(":", maxsplit=1)[-1]
    text = text.strip()
    return {
        OUTPUT_SHORT_ANSWER: text
    }

#### Full chain

In [ ]:
#| export
@dataclass
class LengthConfig:
    """
    Fallacy detector text length configuration
    """
    length_function: Callable[[str], int] # Text length getter
    max_messages_length: int = 2048 # Max history size
    max_fallacies_length: int = 4096 # Max fallacy list representation size


def build_fallacy_detection_chain(llm: BaseLLM, lengths: LengthConfig) -> RunnableSequence:
    """
    Build a sequential chain invoking fallacy detection
    :param llm: Language model to use inside fallacy detector (like ChatOpenAI)
    :param lengths: Fallacy detector text length configuration
    :returns: Sequential chain consuming message history and returning LLM output (and extracted short answer).
    """
    def _stringify_transform(row: dict) -> dict:
        return _stringify(
            row,
            lengths.length_function,
            lengths.max_fallacies_length,
            lengths.max_messages_length,
        )

    stringify_transform = TransformChain(
        input_variables=[INPUT_FALLACIES, INPUT_HISTORY],
        output_variables=[INTERMEDIATE_FALLACIES_STR, INTERMEDIATE_HISTORY_STR],
        transform=_stringify_transform,
    )
    extract_last_user_transform = TransformChain(
        input_variables=[INPUT_HISTORY],
        output_variables=[INTERMEDIATE_LAST_AUTHOR],
        transform=_extract_last_user,
    )
    extract_answer_transform = TransformChain(
        input_variables=[OUTPUT_LLM_OUTPUT],
        output_variables=[OUTPUT_SHORT_ANSWER],
        transform=_extract_answer_from_cot,
    )
    return stringify_transform | \
        extract_last_user_transform | \
        chat_prompt | \
        llm | \
        extract_answer_transform

### Example

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-4-0613",
    openai_api_key=OPENAI_API_KEY,
    streaming=True,
)
encoding = tiktoken.encoding_for_model(llm.model_name)
fallacies_detection_chain = build_fallacy_detection_chain(
    llm,
    LengthConfig(
        lambda text: len(encoding.encode(text)),
        max_messages_length=2048,
        max_fallacies_length=2048 + 1024,
    )
)

In [ ]:
fallacies = read_fallacies(FALLACIES_FNAME)
history = [
    Message("Moonlight", datetime.now(),
            "Soon we will finish with Ukraine"),
    Message("alex4321", datetime.now(),
            "After six months of taking Bakhmut, did anything new happen?\n\n" + \
                "Well, so that there is a reason to suspect that it will happen soon, " + \
                "and not something regardless of the outcome - this will last for years."),
    Message("Moonlight", datetime.now(),
            "Time is a resource, we have plenty of it")
]
context = "Post about the war between Russia/Ukraine"
query = "Moonlight's argument about time being a resource in war"

In [ ]:
fallacies_detection_chain.invoke({
    "fallacies": fallacies,
    "history": history,
    "context": context,
    "query": query,
})

{'llm_output': AIMessageChunk(content='- Possible Fallacies in Moonlight\'s messages:\n  - Argumentum ad Ignorantiam (Appeal to Ignorance):\n    Moonlight could be asserting that Russia will eventually finish with Ukraine because no evidence has been presented to prove otherwise. However, this might not be the case as Moonlight\'s statement could also be seen as a prediction rather than a claim of inevitable truth.\n  - Hasty Generalization:\n    Moonlight might be making a hasty generalization by asserting that time is a resource that Russia has in abundance. This ignores other potential factors that could influence the outcome of the conflict, such as resource limitations, international pressure, or changes in strategy.\n  - Argumentum ad Baculum (Appeal to Force):\n    There\'s a potential for an appeal to force if Moonlight\'s statement "Soon we will finish with Ukraine" is implying that Russia\'s military might will inevitably lead to victory. However, this is a bit of a stretch a

In [ ]:
await fallacies_detection_chain.ainvoke({
    "fallacies": fallacies,
    "history": history,
    "context": context,
    "query": query,
})

TransformChain's atransform is not provided, falling back to synchronous transform


{'llm_output': AIMessageChunk(content='- Possible Fallacies in Moonlight\'s messages:\n  - Argumentum ad Ignorantiam (Appeal to Ignorance):\n    This fallacy could potentially be present in Moonlight\'s messages. In their final statement, they claim that "Time is a resource, we have plenty of it." This could be interpreted as an appeal to ignorance, as they\'re implying that simply having time on their side means they will achieve their goal, without providing evidence to support this claim.\n  - Argumentum ad Populum (Appeal to Popular Opinion):\n    This fallacy is not present in Moonlight\'s messages. They don\'t make any references to popular opinion or majority belief in their statements.\n  - Argumentum ad Hominem (Personal Attack):\n    This fallacy is not present. Moonlight does not attack anyone\'s character in their statements.\n- Therefore, the answer is:\n  - Identified Fallacy 1: Argumentum ad Ignorantiam (Appeal to Ignorance)\n    Moonlight\'s assertion that time is plent

In [ ]:
import nbdev; nbdev.nbdev_export()